In [33]:
import random
"""
point[index, (x,y)]
layout[index, place]
"""
point=[]
layout=[]
minX, maxX = 60, 180
minY, maxY = 16, 72
filename = "./us_cities_1158_afterprocess.xyn.txt"

In [8]:
def prepare_data(select):
    point.clear()
    layout.clear()
    num = 1
    for line in open(filename):
        # print(line)
        index, x, y = line.strip().split(" ")
        if int(index) % select == 0:
            point.extend([(num,(float(x),float(y)))])
            num = num + 1
    for i in range(len(point)):
        layout.append([i+1, random.randint(1,4), 0])
    # print(layout[:])
    # print(point[:])

In [9]:
prepare_data(7)

In [10]:
gapx, gapy = 5, 0.2
def getPos(i,m):
    pointx = point[i][1][0]
    pointy = point[i][1][1]
    # print(pointx, pointy)
    if m==1:
        return [(pointx,pointy),(pointx+gapx,pointy),(pointx+gapx,pointy+gapy),(pointx,pointy+gapy)]
    elif m==2:
        return [(pointx-gapx,pointy),(pointx,pointy),(pointx,pointy+gapy),(pointx-gapx,pointy+gapy)]
    elif m==3:
        return [(pointx-gapx,pointy-gapy),(pointx,pointy-gapy),(pointx,pointy),(pointx-gapx,pointy)]
    else:
        return [(pointx, pointy-gapy),(pointx+gapx,pointy-gapy),(pointx+gapx,pointy),(pointx,pointy)]
    
print(getPos(1,2))

[(90.38, 45.87), (95.38, 45.87), (95.38, 46.07), (90.38, 46.07)]


In [11]:
def inner(point, rec):
    if point[0] > rec[0][0] and point[1] > rec[0][1] and point[0] < rec[1][0] and point[1] > rec[1][1] and point[0] < rec[2][0] and point[1] < rec[2][1] and point[0] > rec[3][0] and point[1] < rec[3][1]:
        return True
    return False

In [12]:
def conflict(vect,i,j):
    """
    判断冲突
    """
    m = vect[i][1]
    n = vect[j][1]
    reci = getPos(i,m)
    recj = getPos(j,n)
    for i in range(len(reci)):
        if inner(reci[i], recj):
            return True
    return False


In [13]:
def costconfilct(vect):
    """
    计算每个点冲突
    """
    sum = 0
    for i in range(len(point)):
        temp = 0
        for j in range(len(point)):
            if i != j:
                if(conflict(vect,i,j)):
                    temp = temp + 1
        vect[i][2] = temp
        sum = sum + temp
    # print("conflict:"+str(sum))
    return sum

# print(costroad(layout))

In [14]:
def cost_function(vect):
    res = 0
    for i in range(len(point)):
        res = res + vect[i][1]*0.1
    return res
print(cost_function(layout))

40.90000000000001


In [15]:
def costroad(vect):
    """
    计算每个点cost
    """
    conflict = costconfilct(vect)
    conflict = 3*conflict + cost_function(vect)
    # print(conflict)
    return conflict
    # sum = 0
    # for i in range(len(point)):
    #     temp = 0
    #     for j in range(len(point)):
    #         if i != j:
    #             if(conflict(vect,i,j)):
    #                 temp = temp + 1
    #     vect[i][2] = temp
    #     sum = sum + temp
    # print("cost:"+str(sum))
    # return sum
costroad(layout)

322.90000000000003

In [16]:
def costrod_lite(new_set, old_set, pointx):
    conflict_sum = 0
    for i in range(len(point)):
        a = conflict(new_set, i, pointx-1)
        b = conflict(old_set, i, pointx-1)
        # print(i,pointx,a,b)
        if a is True and b is False:
            # conflict_sum = conflict_sum + 1
            new_set[i][2] = new_set[i][2]+1
        elif a is False and b is True:
            # conflict_sum = conflict_sum - 1
            new_set[i][2] = new_set[i][2]-1
    # print(new_set[pointx-1][1], old_set[pointx-1][1])
    for i in range(len(point)):
        conflict_sum = conflict_sum + new_set[i][2]
    # print("lite")
    # print(conflict_sum)
    conflict_sum = 3*conflict_sum + cost_function(new_set) - new_set[pointx-1][1]*0.1 + old_set[pointx-1][1]*0.1
    return conflict_sum

In [17]:
import matplotlib.pyplot as plt
def plot(layout,name):
        def create_rect(index, width, height):
                pointx = point[i][1][0]
                pointy = point[i][1][1]
                m = layout[i][1]
                if m==1:
                        x,y = pointx,pointy
                elif m==2:
                        x, y = pointx-width,pointy
                elif m==3:
                        x, y = pointx-width,pointy-height
                else:
                        x, y = pointx, pointy-height
                rect=plt.Rectangle(
                        (x, y),  # (x,y)矩形左下角
                        width,  # width长
                        height,  # height宽
                        # color='maroon', 
                        fill = False
                        )
                return rect
        fig1 = plt.figure(figsize=(8,20))
        plt.axis('off')
        ax1 = fig1.add_subplot(111)

        for i in range(len(point)):
                ax1.add_patch(create_rect(i, gapx, gapy))
        plt.xlim(minX,maxX)
        plt.ylim(minY,maxY)
        plt.savefig("./result_TABU/resultImg-{}.svg".format(name), format="svg")
        plt.show()
# plot()
# plt.savefig('scatter.eps',dpi=600,format='eps')

In [18]:
def plot_withname(a,name="test",select=5):
        point_plot = []
        px = []
        py = []
        index = 1
        filename_plot = "./us_cities_1158_afterprocess.xyn_withname.txt"
        for line in open(filename_plot):
                # print(line)
                if index % select == 0:
                        text, x, y = line.strip().split("*")
                        point_plot.extend([(text,(float(x),float(y)))])
                index = index+1
        for i in range(len(point_plot)):
                px.append(point[i][1][0])
                py.append(point[i][1][1])
        def text(i, width, height):
                pointx = point_plot[i][1][0]
                pointy = point_plot[i][1][1]
                m = a[i][1]
                text = point_plot[i][0]
                if m==1:
                        x,y = pointx,pointy
                elif m==2:
                        x, y = pointx-0.45*len(text),pointy
                elif m==3:
                        x, y = pointx-0.45*len(text),pointy-0.17
                else:
                        x, y = pointx, pointy-0.17
                plt.text(
                        x = x, # x 相对于原先的数据点向左偏移0.12个单位
                        y = y, # y 向上偏移0.05个单位
                        s = '{}'.format(text), # 数据格式
                        fontsize=3,
                        color ='r') # 颜色
                # return t
        fig1 = plt.figure(figsize=(8,20))
        plt.axis('off')
        ax1 = fig1.add_subplot(111)

        for i in range(len(point)):
                text(i, gapx, gapy)
        plt.xlim(60,180)
        plt.ylim(16,72)
        plt.scatter(px,py,s=1)
        plt.savefig("./result_TABU/{}.svg".format(name), format="svg")
        plt.show()

In [37]:
import copy
def tabusearch(iteration_times,cacu_time,tabu_length,origin_times,costf,printf,costflite,namefortest):
    """
    iteration_times为每一个初始解的迭代次数，cacu_time为候选集合长度，tabu_length为禁忌长度，
    origin_times为整个程序循环次数，可以理解为使用不同个初始解，costf为成本函数，printtravel为打印结果函数
    """
    def pan_move(move_step,tabu_move):  #判断移动是否在禁忌区域中，如果是返回True和该点索引，否则返回False和0
        if move_step in tabu_move:
            index = tabu_move.index(move_step)
            return(True,index)
        else:
            return(False,0)
    def pan_cost(cost,tabu_cost,t):  #判断该移动是否比禁忌区域中该移动小，如果小则返回True，否则返回False
        if cost<tabu_cost[t]:
            return(True)
        else:
            return(False)  
    def add_tabu(cost,move,tabu_cost,tabu_move,t):  #为禁忌区域添加移动和成本，若超过T则剔除最先进入的禁忌
        tabu_cost.append(cost)
        tabu_move.append(move)
        if len(tabu_cost)>t:
            del tabu_cost[0]
        if len(tabu_move)>t:
            del tabu_move[0]
        return(tabu_cost,tabu_move)
    def cacu(vec,t):    #为每一个初始解计算t次
        vec_set = []
        m_set = []
        cost_set = []
        h = []
        for i in range(t):
            v,m,c,h = move(vec,h)
            vec_set.append(v)
            m_set.append(m)
            cost_set.append(c)
        return(vec_set,m_set,cost_set)
    def cacu_tiqu(v1,m1,c1):    #从上述t次筛选最小的解向量，移动和成本
        t = c1.index(min(c1))
        v_max = v1[t]
        m_max = m1[t]
        c_max = c1[t]
        return(v_max,m_max,c_max)
    def move(vec,h):  #输出移动后的向量，和成本
        i = 1
        while i==1:
            sel = random.sample(vec,1)
            # print(sel)
            # m.sort()
            while True:
                nextPos = random.randint(1,4)
                if nextPos != sel[0][1]:
                    break
            m = [sel[0][0],sel[0][1],nextPos]
            if m not in h:
                h.append(m)
                vec_copy = copy.deepcopy(vec)
                vec_copy[sel[0][0]-1][1] = m[2]
                # vec_copy[vec_copy.index(m)][2] = cal
                # vec_copy[vec_copy.index(m[1])] = m[0]
                # print("vec是：")
                # print((vec_copy))
                cost = costflite(vec_copy, vec, m[0])
                # print("损失是："+str(cost))
                i = 0
                return(vec_copy,m,cost,h)
    finall_road = []
    finall_cost = []
    layout = []
    for t1 in range(origin_times):
        # road = [i for i in range(1,29)]
        # random.shuffle(road)
        layout.clear()
        for i in range(len(point)):
            layout.append([i+1, random.randint(1,4), 0])
        print("初始状态：")
        cost_init = costf(layout)
        confilct_init = costconfilct(layout)
        print(cost_init)
        nameprefix = str(namefortest) + str(confilct_init)
        # plot(layout=layout,name=nameprefix)
        # plot_withname(a=layout,name=nameprefix,select=sel)
        
        # print(costf(layout))
        # plot(layout)
        # fig1 = plt.figure()
        # ax1 = fig1.add_subplot(111)
        # for i in range(len(point)):
        #     ax1.add_patch(create_rect(i, gapx, gapy))
        # plt.xlim(2600,5000)
        # plt.ylim(7100,12312)
        # plt.show()

        tabu_cost = []
        tabu_move = []
        tabu_length = 7 + int(costconfilct(layout)*0.25)
        cacu_time = 1 + int(costconfilct(layout)*0.05)
        print("计算次数："+str(cacu_time))
        print("tabu长度："+str(tabu_length))
        for t in range(iteration_times):
            if t % 50 == 0:
                cacu_time = 1 + int(costconfilct(layout)*0.05)
                tabu_length = 7 + int(costconfilct(layout)*0.25)
                print("循环次数："+str(t)+"计算次数："+str(cacu_time))
                print("循环次数："+str(t)+"tabu长度："+str(tabu_length))
            i = 0
            while i==0:
                v1,m1,c1 = cacu(layout,cacu_time)
                v_m,m_m,c_m = cacu_tiqu(v1,m1,c1)
                key1 = pan_move(m_m,tabu_move)
                if key1[0]:
                    print("enter!")
                    if pan_cost(c_m,tabu_cost,key1[1]):
                        layout = v_m
                        finall_road.append(layout)
                        print("costfunc:"+str(c_m))
                        finall_cost.append(c_m)
                        tabu_cost,tabu_move = add_tabu(c_m,m_m,tabu_cost,tabu_move,tabu_length)
                        i=1
                    else:
                        v1.remove(v_m)
                        m1.remove(m_m)
                        c1.remove(c_m)
                        if len(v1)==0:
                            i = 1
                else:
                    # print("costfunc:"+str(c_m))
                    tabu_cost,tabu_move = add_tabu(c_m,m_m,tabu_cost,tabu_move,tabu_length)
                    layout = v_m
                    finall_road.append(layout)
                    finall_cost.append(c_m)
                    i = 1
    index = finall_cost.index(min(finall_cost))
    # s2 = datetime.datetime.now()
    # plot(finall_road[index])
    # return(finall_road[index],min(finall_cost),printf(finall_road[index]),costconfilct(layout))
    return(finall_road[index],min(finall_cost),costconfilct(layout), cost_init, confilct_init)


In [ ]:
# tabusearch(iteration_times=1000,cacu_time=10,tabu_length=10,origin_times=1,costf=costroad,printf=print)
# # plot()

In [ ]:
prepare_data(7)
test1_name1 = "500_10_10_165_random"
test1_name2 = "500_10_10_165_result"
a,b,c,d = tabusearch(sel=7,iteration_times=500,cacu_time=10,tabu_length=10,origin_times=1,costf=costroad,printf=print,costflite=costrod_lite,namefortest=test1_name1)
# plt.plot(result_record)
# plt.show()
cc = costroad(a)
print(cc)
nameprefix = str(test1_name2) + str(cc)
plot(layout=a,name=nameprefix)
plot_withname(a=a,name=nameprefix,select=7)
# plt.plot(result_record)
# plt.show()

In [ ]:
prepare_data(5)
test1_name1 = "500_10_10_231_random"
test1_name2 = "500_10_10_231_result"
a,b,c,d = tabusearch(sel=5,iteration_times=500,cacu_time=10,tabu_length=10,origin_times=1,costf=costroad,printf=print,costflite=costrod_lite,namefortest=test1_name1)
# plt.plot(result_record)
# plt.show()
cc = costroad(a)
print(cc)
nameprefix = str(test1_name2) + str(cc)
plot(layout=a,name=nameprefix)
plot_withname(a=a,name=nameprefix,select=5)
# plt.plot(result_record)
# plt.show()

In [ ]:
prepare_data(5)
test1_name1 = "1000_10_10_231_random"
test1_name2 = "1000_10_10_231_result"
a,b,c,d = tabusearch(iteration_times=1000,cacu_time=10,tabu_length=10,origin_times=1,costf=costroad,printf=print,costflite=costrod_lite,namefortest=test1_name1)
# plt.plot(result_record)
# plt.show()
cc = costroad(a)
print(cc)
nameprefix = str(test1_name2) + str(cc)
plot(layout=a,name=nameprefix)
# plt.plot(result_record)
# plt.show()

In [ ]:
prepare_data(4)
test1_name1 = "500_10_10_289_random"
test1_name2 = "500_10_10_289_result"
a,b,c,d = tabusearch(iteration_times=500,cacu_time=10,tabu_length=10,origin_times=1,costf=costroad,printf=print,costflite=costrod_lite,namefortest=test1_name1)
# plt.plot(result_record)
# plt.show()
cc = costroad(a)
print(cc)
nameprefix = str(test1_name2) + str(cc)
plot(layout=a,name=nameprefix)
# plt.plot(result_record)
# plt.show()

In [ ]:
prepare_data(4)
test1_name1 = "1000_10_10_289_random"
test1_name2 = "1000_10_10_289_result"
a,b,c,d = tabusearch(iteration_times=1000,cacu_time=10,tabu_length=10,origin_times=1,costf=costroad,printf=print,costflite=costrod_lite,namefortest=test1_name1)
# plt.plot(result_record)
# plt.show()
cc = costroad(a)
print(cc)
nameprefix = str(test1_name2) + str(cc)
plot(layout=a,name=nameprefix)
# plt.plot(result_record)
# plt.show()

In [ ]:
prepare_data(3)
test1_name1 = "500_10_10_386_random"
test1_name2 = "500_10_10_386_result"
a,b,c,d = tabusearch(iteration_times=500,cacu_time=10,tabu_length=10,origin_times=1,costf=costroad,printf=print,costflite=costrod_lite,namefortest=test1_name1)
# plt.plot(result_record)
# plt.show()
cc = costroad(a)
print(cc)
nameprefix = str(test1_name2) + str(cc)
plot(layout=a,name=nameprefix)
# plt.plot(result_record)
# plt.show()

In [ ]:
prepare_data(3)
test1_name1 = "1000_10_10_386_random"
test1_name2 = "1000_10_10_386_result"
a,b,c,d = tabusearch(iteration_times=1000,cacu_time=10,tabu_length=10,origin_times=1,costf=costroad,printf=print,costflite=costrod_lite,namefortest=test1_name1)
# plt.plot(result_record)
# plt.show()
cc = costroad(a)
print(cc)
nameprefix = str(test1_name2) + str(cc)
plot(layout=a,name=nameprefix)
# plt.plot(result_record)
# plt.show()

In [ ]:
prepare_data(2)
test1_name1 = "1000_10_10_579_random"
test1_name2 = "1000_10_10_579_result"
a,b,c,d = tabusearch(iteration_times=1000,cacu_time=10,tabu_length=10,origin_times=1,costf=costroad,printf=print,costflite=costrod_lite,namefortest=test1_name1)
# plt.plot(result_record)
# plt.show()
cc = costroad(a)
print(cc)
nameprefix = str(test1_name2) + str(cc)
plot(layout=a,name=nameprefix)
# plt.plot(result_record)
# plt.show()

In [ ]:
prepare_data(2)
test1_name1 = "500_10_10_579_random"
test1_name2 = "500_10_10_579_result"
a,b,c,d = tabusearch(iteration_times=500,cacu_time=10,tabu_length=10,origin_times=1,costf=costroad,printf=print,costflite=costrod_lite,namefortest=test1_name1)
# plt.plot(result_record)
# plt.show()
cc = costroad(a)
print(cc)
nameprefix = str(test1_name2) + str(cc)
plot(layout=a,name=nameprefix)
# plt.plot(result_record)
# plt.show()

# 不同的数据集大小进行性能比较

In [40]:
import random
import time
import os
indexAll = range(1,1159)
savefolder = "./result_tabu_differentscale"

In [41]:
def save(layout, cost_result, time, path):
        #gapx, gapy = 20, 8
# W1, W2 = 3, 1
# minX, maxX = -20, 500
# minY, maxY = -20, 500
        f = open(path, "w")
        f.write("Process Time: {}\n".format(time))
        f.write("cost_init,cost_final,confilct_init,conflict_final\n")
        for i in range(len(cost_result)):
                if i == len(cost_result)-1:
                        f.write("{}\n".format(cost_result[i]))
                else:
                        f.write("{} ".format(cost_result[i]))
        f.write("gapx, gapy = {},{}\n".format(gapx, gapy))
        f.write("minX, maxX = {},{}\n".format(minX, maxX))
        f.write("minY, maxY = {},{}\n".format(minY, maxY))
        for i in range(len(layout)):
                towrite = str(point[i][1][0]) + " " + str(point[i][1][1]) + " " + str(layout[i][1]) + "\n"
                f.write(towrite)
        f.close()

In [42]:
def dataselect(volume):
    point.clear()
    layout.clear()
    index1 = random.sample(indexAll, volume) 
    num = 1
    for line in open(filename):
        # print(line)
        index, x, y = line.strip().split(" ")
        if int(index) in index1:
            point.extend([(num,(float(x),float(y)))])
            num = num + 1
    for i in range(len(point)):
        layout.append([i+1, random.randint(1,4), 0])

In [43]:
dataselect(100)
print(len(point))

100


In [44]:
for i in range(100,1200,100):
    dataselect(i)
    iteration = 1500
    saveinfo = str(i) + "_" + "iteration" + " " + str(iteration) 
    print("{} started".format(i/100))
    tic = time.time()
    a,b,c,cost_init,confilct_init = tabusearch(iteration_times=iteration,cacu_time=10,tabu_length=10,origin_times=1,costf=costroad,printf=print,costflite=costrod_lite,namefortest=filename)
    toc = time.time()
    processTime = toc-tic
    cost_final = costroad(a)
    confilct_final = costconfilct(a)
    cost_result = []
    cost_result.append(cost_init)
    cost_result.append(cost_final)
    cost_result.append(confilct_init)
    cost_result.append(confilct_final)
    print(cost_final, confilct_final)
    # plot(layout=a,name=filelist[i])
    savefilename = os.path.join(savefolder, saveinfo)
    save(a, cost_result, processTime, savefilename)
    print("{} finished".format(i/100))

1.0 started
初始状态：
86.1
计算次数：2
tabu长度：12
循环次数：0计算次数：2
循环次数：0tabu长度：12
循环次数：50计算次数：1
循环次数：50tabu长度：10
循环次数：100计算次数：1
循环次数：100tabu长度：11
循环次数：150计算次数：2
循环次数：150tabu长度：12
循环次数：200计算次数：1
循环次数：200tabu长度：9
循环次数：250计算次数：2
循环次数：250tabu长度：14
循环次数：300计算次数：1
循环次数：300tabu长度：9
循环次数：350计算次数：2
循环次数：350tabu长度：14
循环次数：400计算次数：1
循环次数：400tabu长度：9
循环次数：450计算次数：2
循环次数：450tabu长度：12
循环次数：500计算次数：1
循环次数：500tabu长度：10
循环次数：550计算次数：2
循环次数：550tabu长度：12
enter!
costfunc:75.99999999999997
循环次数：600计算次数：1
循环次数：600tabu长度：10
循环次数：650计算次数：2
循环次数：650tabu长度：13
循环次数：700计算次数：1
循环次数：700tabu长度：11
循环次数：750计算次数：2
循环次数：750tabu长度：15
循环次数：800计算次数：1
循环次数：800tabu长度：11
循环次数：850计算次数：2
循环次数：850tabu长度：15
enter!
循环次数：900计算次数：1
循环次数：900tabu长度：11
循环次数：950计算次数：2
循环次数：950tabu长度：13
循环次数：1000计算次数：1
循环次数：1000tabu长度：11
循环次数：1050计算次数：2
循环次数：1050tabu长度：12
循环次数：1100计算次数：1
循环次数：1100tabu长度：9
循环次数：1150计算次数：1
循环次数：1150tabu长度：11
循环次数：1200计算次数：2
循环次数：1200tabu长度：12
循环次数：1250计算次数：1
循环次数：1250tabu长度：9
循环次数：1300计算次数：2
循环次数：1300tabu长度：13
enter!
循环次数：1350计算次数：1
循环

In [45]:
for i in range(100,1200,100):
    dataselect(i)
    iteration = 1000
    saveinfo = str(i) + "_" + "iteration" + " " + str(iteration) 
    print("{} started".format(i/100))
    tic = time.time()
    a,b,c,cost_init,confilct_init = tabusearch(iteration_times=iteration,cacu_time=10,tabu_length=10,origin_times=1,costf=costroad,printf=print,costflite=costrod_lite,namefortest=filename)
    toc = time.time()
    processTime = toc-tic
    cost_final = costroad(a)
    confilct_final = costconfilct(a)
    cost_result = []
    cost_result.append(cost_init)
    cost_result.append(cost_final)
    cost_result.append(confilct_init)
    cost_result.append(confilct_final)
    print(cost_final, confilct_final)
    # plot(layout=a,name=filelist[i])
    savefilename = os.path.join(savefolder, saveinfo)
    save(a, cost_result, processTime, savefilename)
    print("{} finished".format(i/100))

1.0 started
初始状态：
145.10000000000002
计算次数：3
tabu长度：17
循环次数：0计算次数：3
循环次数：0tabu长度：17
循环次数：50计算次数：1
循环次数：50tabu长度：10
循环次数：100计算次数：1
循环次数：100tabu长度：11
循环次数：150计算次数：2
循环次数：150tabu长度：15
循环次数：200计算次数：2
循环次数：200tabu长度：12
循环次数：250计算次数：1
循环次数：250tabu长度：8
循环次数：300计算次数：2
循环次数：300tabu长度：12
循环次数：350计算次数：2
循环次数：350tabu长度：12
循环次数：400计算次数：1
循环次数：400tabu长度：9
循环次数：450计算次数：2
循环次数：450tabu长度：15
循环次数：500计算次数：1
循环次数：500tabu长度：11
循环次数：550计算次数：2
循环次数：550tabu长度：16
循环次数：600计算次数：2
循环次数：600tabu长度：12
循环次数：650计算次数：1
循环次数：650tabu长度：11
enter!
循环次数：700计算次数：2
循环次数：700tabu长度：14
循环次数：750计算次数：1
循环次数：750tabu长度：11
循环次数：800计算次数：2
循环次数：800tabu长度：14
循环次数：850计算次数：1
循环次数：850tabu长度：10
循环次数：900计算次数：1
循环次数：900tabu长度：11
循环次数：950计算次数：2
循环次数：950tabu长度：13
38.6 4
1.0 finished
2.0 started
初始状态：
508.79999999999995
计算次数：8
tabu长度：45
循环次数：0计算次数：8
循环次数：0tabu长度：45
循环次数：50计算次数：4
循环次数：50tabu长度：25
循环次数：100计算次数：3
循环次数：100tabu长度：19
循环次数：150计算次数：3
循环次数：150tabu长度：18
循环次数：200计算次数：2
循环次数：200tabu长度：16
enter!
循环次数：250计算次数：3
循环次数：250tabu长度：20
循环次数：300计算次数：3

In [46]:
for i in range(100,1200,100):
    dataselect(i)
    iteration = 500
    saveinfo = str(i) + "_" + "iteration" + " " + str(iteration) 
    print("{} started".format(i/100))
    tic = time.time()
    a,b,c,cost_init,confilct_init = tabusearch(iteration_times=iteration,cacu_time=10,tabu_length=10,origin_times=1,costf=costroad,printf=print,costflite=costrod_lite,namefortest=filename)
    toc = time.time()
    processTime = toc-tic
    cost_final = costroad(a)
    confilct_final = costconfilct(a)
    cost_result = []
    cost_result.append(cost_init)
    cost_result.append(cost_final)
    cost_result.append(confilct_init)
    cost_result.append(confilct_final)
    print(cost_final, confilct_final)
    # plot(layout=a,name=filelist[i])
    savefilename = os.path.join(savefolder, saveinfo)
    save(a, cost_result, processTime, savefilename)
    print("{} finished".format(i/100))

1.0 started
初始状态：
110.0
计算次数：2
tabu长度：14
循环次数：0计算次数：2
循环次数：0tabu长度：14
循环次数：50计算次数：1
循环次数：50tabu长度：10
循环次数：100计算次数：2
循环次数：100tabu长度：12
循环次数：150计算次数：1
循环次数：150tabu长度：8
enter!
循环次数：200计算次数：1
循环次数：200tabu长度：11
循环次数：250计算次数：1
循环次数：250tabu长度：11
循环次数：300计算次数：2
循环次数：300tabu长度：12
循环次数：350计算次数：1
循环次数：350tabu长度：10
循环次数：400计算次数：2
循环次数：400tabu长度：12
循环次数：450计算次数：1
循环次数：450tabu长度：10
45.69999999999998 6
1.0 finished
2.0 started
初始状态：
380.0
计算次数：6
tabu长度：34
循环次数：0计算次数：6
循环次数：0tabu长度：34
循环次数：50计算次数：3
循环次数：50tabu长度：19
循环次数：100计算次数：3
循环次数：100tabu长度：18
循环次数：150计算次数：2
循环次数：150tabu长度：14
循环次数：200计算次数：2
循环次数：200tabu长度：16
循环次数：250计算次数：3
循环次数：250tabu长度：19
循环次数：300计算次数：3
循环次数：300tabu长度：17
循环次数：350计算次数：2
循环次数：350tabu长度：14
循环次数：400计算次数：3
循环次数：400tabu长度：17
循环次数：450计算次数：2
循环次数：450tabu长度：16
141.8 30
2.0 finished
3.0 started
初始状态：
946.9
计算次数：15
tabu长度：79
循环次数：0计算次数：15
循环次数：0tabu长度：79
循环次数：50计算次数：9
循环次数：50tabu长度：49
循环次数：100计算次数：7
循环次数：100tabu长度：37
循环次数：150计算次数：5
循环次数：150tabu长度：31
enter!
costfunc:345.49999999999994
循环次数：

In [48]:
for i in [500]:
    dataselect(i)
    iteration = 1000
    saveinfo = str(i) + "_" + "iteration" + " " + str(iteration) 
    print("{} started".format(i/100))
    tic = time.time()
    a,b,c,cost_init,confilct_init = tabusearch(iteration_times=iteration,cacu_time=10,tabu_length=10,origin_times=1,costf=costroad,printf=print,costflite=costrod_lite,namefortest=filename)
    toc = time.time()
    processTime = toc-tic
    cost_final = costroad(a)
    confilct_final = costconfilct(a)
    cost_result = []
    cost_result.append(cost_init)
    cost_result.append(cost_final)
    cost_result.append(confilct_init)
    cost_result.append(confilct_final)
    print(cost_final, confilct_final)
    # plot(layout=a,name=filelist[i])
    savefilename = os.path.join(savefolder, saveinfo)
    save(a, cost_result, processTime, savefilename)
    print("{} finished".format(i/100))

5.0 started
初始状态：
2850.1
计算次数：46
tabu长度：234
循环次数：0计算次数：46
循环次数：0tabu长度：234
循环次数：50计算次数：31
循环次数：50tabu长度：160
循环次数：100计算次数：24
循环次数：100tabu长度：123
循环次数：150计算次数：19
循环次数：150tabu长度：99
循环次数：200计算次数：16
循环次数：200tabu长度：86
循环次数：250计算次数：15
循环次数：250tabu长度：78
循环次数：300计算次数：14
循环次数：300tabu长度：72
循环次数：350计算次数：13
循环次数：350tabu长度：71
enter!
costfunc:892.0000000000002
enter!
costfunc:881.2
循环次数：400计算次数：13
循环次数：400tabu长度：67
循环次数：450计算次数：12
循环次数：450tabu长度：64
enter!
costfunc:810.1000000000001
循环次数：500计算次数：12
循环次数：500tabu长度：63
enter!
costfunc:791.7000000000002
enter!
costfunc:791.8000000000002
enter!
costfunc:791.9000000000002
循环次数：550计算次数：12
循环次数：550tabu长度：62
enter!
costfunc:777.7
enter!
costfunc:777.7
enter!
循环次数：600计算次数：11
循环次数：600tabu长度：61
enter!
costfunc:770.8000000000001
enter!
costfunc:771.0
enter!
costfunc:771.1
循环次数：650计算次数：11
循环次数：650tabu长度：60
enter!
costfunc:757.9999999999999
循环次数：700计算次数：11
循环次数：700tabu长度：60
enter!
costfunc:753.8000000000001
enter!
costfunc:757.6
循环次数：750计算次数：11
循环次数：750tabu长度：60
ente